In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("spec_classification")
umod = UniversalModeler()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [4]:
factors = []
factors.append("d1")
factors.append("d2")
factors.append("d3")
factors.append("rolling14")

In [5]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
training_sets = []

In [8]:
for ticker in sp500["ticker"].unique():
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
        ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
        ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
        ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
        ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["future"] = ticker_data["adjclose"].shift(-1)
        ticker_data["delta"] = (ticker_data["future"] - ticker_data["adjclose"]) / ticker_data["adjclose"]
        ticker_data["y"] = [x > 0 for x in ticker_data["delta"]]
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except:
        continue

In [9]:
data = pd.concat(training_sets)

In [10]:
data = data.dropna()

In [11]:
data

,year,week,ticker,adjclose,y,d1,d2,d3,rolling14
13,2000,14,MMM,25.077294,True,0.041247,12.190637,-13.811506,24.512802
14,2000,15,MMM,25.480959,False,0.016097,-0.609744,-1.050017,24.464388
15,2000,16,MMM,24.647652,True,-0.032703,-3.031650,3.972003,24.279800
16,2000,17,MMM,24.709535,False,0.002511,-1.076772,-0.644823,24.181488
17,2000,18,MMM,23.492011,False,-0.049273,-20.625452,18.154890,24.048622
...,...,...,...,...,...,...,...,...,...
523,2023,5,ZTS,167.360000,False,0.006289,-0.804107,8.300468,151.412486
524,2023,6,ZTS,162.342000,True,-0.029983,-5.767340,6.172354,152.808105
525,2023,7,ZTS,171.204000,False,0.054588,-2.820631,-0.510930,155.082868
526,2023,8,ZTS,168.025000,False,-0.018568,-1.340154,-0.524874,156.626157


In [12]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2019,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.classification_model(training_slice,prediction_set,factors,multioutput=False)
        stuff["training_years"] = years
        spec.store(set_name,stuff)
spec.disconnect()

  0%|                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

Epoch 1/10
2336/2336 [==============================] - 2s 800us/step - loss: 378774816.0000 - accuracy: 0.4711
Epoch 2/10
2336/2336 [==============================] - 2s 776us/step - loss: 297853344.0000 - accuracy: 0.4510
Epoch 3/10
2336/2336 [==============================] - 2s 774us/step - loss: 137540016.0000 - accuracy: 0.4495
Epoch 4/10
2336/2336 [==============================] - 2s 788us/step - loss: 73050096.0000 - accuracy: 0.4480
Epoch 5/10
2336/2336 [==============================] - 2s 811us/step - loss: 301509024.0000 - accuracy: 0.4482
Epoch 6/10
2336/2336 [==============================] - 2s 798us/step - loss: 4549069.0000 - accuracy: 0.4470
Epoch 7/10
2336/2336 [==============================] - 2s 783us/step - loss: 344445536.0000 - accuracy: 0.4466
Epoch 8/10
2336/2336 [==============================] - 2s 783us/step - loss: 248635744.0000 - accuracy: 0.4470
Epoch 9/10
2336/2336 [==============================] - 2s 779us/step - loss: 258927968.0000 - accuracy: 0.

 25%|█████████████████████████████████████▎                                                                                                               | 1/4 [04:21<13:04, 261.67s/it]

Epoch 1/10
2347/2347 [==============================] - 2s 808us/step - loss: 143363216.0000 - accuracy: 0.4589
Epoch 2/10
2347/2347 [==============================] - 2s 795us/step - loss: 301600608.0000 - accuracy: 0.4308
Epoch 3/10
2347/2347 [==============================] - 2s 811us/step - loss: 208577936.0000 - accuracy: 0.4311
Epoch 4/10
2347/2347 [==============================] - 2s 799us/step - loss: 135891520.0000 - accuracy: 0.4308
Epoch 5/10
2347/2347 [==============================] - 2s 789us/step - loss: 103907032.0000 - accuracy: 0.4302
Epoch 6/10
2347/2347 [==============================] - 2s 793us/step - loss: 538805.3125 - accuracy: 0.4297
Epoch 7/10
2347/2347 [==============================] - 2s 785us/step - loss: 322292224.0000 - accuracy: 0.4299
Epoch 8/10
2347/2347 [==============================] - 2s 820us/step - loss: 75129080.0000 - accuracy: 0.4296
Epoch 9/10
2347/2347 [==============================] - 2s 800us/step - loss: 25748148.0000 - accuracy: 0.42

 50%|██████████████████████████████████████████████████████████████████████████▌                                                                          | 2/4 [08:45<08:46, 263.18s/it]

Epoch 1/10
2377/2377 [==============================] - 2s 778us/step - loss: 733800896.0000 - accuracy: 0.4835
Epoch 2/10
2377/2377 [==============================] - 2s 763us/step - loss: 872738432.0000 - accuracy: 0.4383
Epoch 3/10
2377/2377 [==============================] - 2s 804us/step - loss: 627023936.0000 - accuracy: 0.4306
Epoch 4/10
2377/2377 [==============================] - 2s 768us/step - loss: 133097136.0000 - accuracy: 0.4300
Epoch 5/10
2377/2377 [==============================] - 2s 766us/step - loss: 523236096.0000 - accuracy: 0.4287
Epoch 6/10
2377/2377 [==============================] - 2s 781us/step - loss: 469709888.0000 - accuracy: 0.4288
Epoch 7/10
2377/2377 [==============================] - 2s 785us/step - loss: 188483360.0000 - accuracy: 0.4283
Epoch 8/10
2377/2377 [==============================] - 2s 792us/step - loss: 69061888.0000 - accuracy: 0.4281
Epoch 9/10
2377/2377 [==============================] - 2s 780us/step - loss: 198925968.0000 - accuracy: 

 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                     | 3/4 [13:10<04:23, 263.66s/it]

Epoch 1/10
2394/2394 [==============================] - 2s 822us/step - loss: 680710336.0000 - accuracy: 0.4802
Epoch 2/10
2394/2394 [==============================] - 2s 797us/step - loss: 279535.5312 - accuracy: 0.4368
Epoch 3/10
2394/2394 [==============================] - 2s 813us/step - loss: 1022502720.0000 - accuracy: 0.4354
Epoch 4/10
2394/2394 [==============================] - 2s 789us/step - loss: 317151744.0000 - accuracy: 0.4339
Epoch 5/10
2394/2394 [==============================] - 2s 818us/step - loss: 289036224.0000 - accuracy: 0.4329
Epoch 6/10
2394/2394 [==============================] - 2s 787us/step - loss: 347794.9688 - accuracy: 0.4336
Epoch 7/10
2394/2394 [==============================] - 2s 777us/step - loss: 446221728.0000 - accuracy: 0.4328
Epoch 8/10
2394/2394 [==============================] - 2s 781us/step - loss: 593642240.0000 - accuracy: 0.4325
Epoch 9/10
2394/2394 [==============================] - 2s 779us/step - loss: 908925952.0000 - accuracy: 0.43

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [17:27<00:00, 261.86s/it]
